## Think Bayes solutions using coppertop

In this notebook we use coppertop to work through some of the examples, problems and excerises from Allen B. Downey's book _Think Bayes_ - a copy of which can be found [here](https://github.com/coppertop-bones/coppertop-libs/blob/main/jupyter/think%20bayes/thinkbayes.pdf).

In the document:

_Permission is granted to copy, distribute, and/or modify this document
under the terms of the Creative Commons Attribution-NonCommercial
3.0 Unported License, which is available at http://creativecommons.org/
licenses/by-nc/3.0/_

See [coppertop-bones/README.md](https://github.com/coppertop-bones/coppertop) for notes on usage of coppertop.

<br>

#### imports and enum definition

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt, numpy as np, enum

In [2]:
from coppertop.pipe import *

from coppertop.dm.core.types import dstruct, dmap, pytuple, pylist, T, T1, T2, T3, num, txt, py
from coppertop.dm.pmf import L, PMF, CMF, formatPmf, pmfMul
import coppertop.dm.pp                          # load pp functions
from _ import *                       # import all loaded uber functions

class E(enum.IntEnum):
    A = enum.auto()
    B = enum.auto()
    C = enum.auto()
    J1 = enum.auto()
    J2 = enum.auto()
    D4 = enum.auto()
    D6 = enum.auto()
    D8 = enum.auto()
    D12 = enum.auto()
    D20 = enum.auto()
    
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name
    
A = E.A
B = E.B
C = E.C
J1 = E.J1
J2 = E.J2
D4 = E.D4
D6 = E.D6
D8 = E.D8
D12 = E.D12
D20 = E.D20

<br>

#### 1.3 The cookie problem

We’ll get to Bayes’s theorem soon, but I want to motivate it with an example
called the cookie problem. Suppose there are two bowls of cookies. Bowl 1
contains 30 vanilla cookies and 10 chocolate cookies. Bowl 2 contains 20 of
each.

Now suppose you choose one of the bowls at random and, without looking,
select a cookie at random. The cookie is vanilla. What is the probability that
it came from Bowl 1?

This is a conditional probability; we want $\mathbf{P}\left(Bowl1 \mathbin{\vert} vanilla\right)$, but it is not
obvious how to compute it. If I asked a different question—the probability
of a vanilla cookie given Bowl 1—it would be easy:

$$
\begin{align}
\mathbf{P}( vanilla\mathbin{\vert}Bowl1) = 3/4
\end{align}
$$

Sadly, $\mathbf{P}\left(A\mathbin{\vert}B\right)$ is not the same as $\mathbf{P}\left(B\mathbin{\vert}A\right)$, but 
there is a way to get from one to the other: Bayes’s theorem.

<br>

#### Bayes Refresher
See _["An Essay towards solving a Problem in the Doctrine of Chances"](https://github.com/coppertop-bones/coppertop-libs/blob/main/jupyter/think%20bayes/article.pdf)_.

from PROP 3

$$
\begin{align}
\mathbf{P}\left(B \cap A\right) = \mathbf{P}\left(B\mathbin{\vert}A\right)\cdot \mathbf{P}\left(A\right)\\
\end{align}
$$

and obviously
$$\mathbf{P}(A \cap B) = \mathbf{P}(B \cap A)$$

so
$$
\begin{align}
\mathbf{P}( A\mathbin{\vert}B) \cdot \mathbf{P}(B)=\mathbf{P}(B\mathbin{\vert}A)\cdot \mathbf{P}(A)
\end{align}
$$

aka
$$
\begin{align}
\mathbf{P}( hypothesis\mathbin{\vert}data) \cdot \mathbf{P}(data)=\mathbf{P}(data\mathbin{\vert}hypothesis)\cdot \mathbf{P}(hypothesis)
\end{align}
$$

Comtemporaneous (look it up) version, i.e. after some data is known
$$
\begin{align}
posterior =likelihood\cdot prior \cdot constant
\end{align}
$$

<br>

#### 1.6 The M&M Problem

M&M’s are small candy-coated chocolates that come in a variety of colors.
Mars, Inc., which makes M&M’s, changes the mixture of colors from time
to time.

In 1995, they introduced blue M&M’s. Before then, the color mix in a bag
of plain M&M’s was 30% Brown, 20% Yellow, 20% Red, 10% Green, 10%
Orange, 10% Tan. Afterward it was 24% Blue , 20% Green, 16% Orange,
14% Yellow, 13% Red, 13% Brown.

Suppose a friend of mine has two bags of M&M’s, and he tells me that one
is from 1994 and one from 1996. He won’t tell me which is which, but he
gives me one M&M from each bag. One is yellow and one is green. What is
the probability that the yellow one came from the 1994 bag?

In [3]:
bag1994 = dstruct(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10)
bag1996 = dstruct(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24)
[bag1994, bag1996] >> collect >> PP;

dstruct(Brown=30, Yellow=20, Red=20, Green=10, Orange=10, Tan=10)
dstruct(Brown=13, Yellow=14, Red=13, Green=20, Orange=16, Blue=24)


hypA -> yellow is from 1994, green is from 1996\
hypB -> green is from 1994, yellow is from 1996

In [4]:
prior = PMF({A:0.5, B:0.5}) >> PP

likelihood = L({
    A: bag1994.Yellow * bag1996.Green, 
    B: bag1994.Green * bag1996.Yellow
}) >> PP

post = prior >> pmfMul >> likelihood >> normalise
post >> PP

20/27

PMF(A: 0.500, B: 0.500)
L(A: 400.000, B: 140.000)
PMF(A: 0.741, B: 0.259)


ponder some more
ponder some more


0.7407407407407407

<br>

#### 1.7 The Monty Hall problem

Monty Hall was the original host of the game show Let’s Make a Deal. The
Monty Hall problem is based on one of the regular games on the show. If
you are on the show, here’s what happens:

• Monty shows you three closed doors and tells you that there is a prize
behind each door: one prize is a car, the other two are less valuable
prizes like peanut butter and fake finger nails. The prizes are arranged
at random.

• The object of the game is to guess which door has the car. If you guess
right, you get to keep the car.

• You pick a door, which we will call Door A. We’ll call the other doors
B and C.

• Before opening the door you chose, Monty increases the suspense by
opening either Door B or C, whichever does not have the car. (If the
car is actually behind Door A, Monty can safely open B or C, so he
chooses one at random.)

• Then Monty offers you the option to stick with your original choice or
switch to the one remaining unopened door.

The question is, should you “stick” or “switch” or does it make no difference?

**Minor reframe**

Let A be the door we initially choose at random \
Let B be the door Monty selects to show us to be without a car \
Let C be the other door we can choose after the fact

In [5]:
prior = PMF({A:1, B:1, C:1}) >> PP
likelihood = L({ # i.e. likelihood of monty opening B given that the car is behind each, i.e. p(data|hyp)
    A: 0.5,      # prob of opening B if behind A - he can choose at random so 50:50
    B: 0,        # prob of opening B if behind B - Monty can't open B else he'd reveal the car, so cannot open B => 0%
    C: 1,        # prob of opening B if behind C - Monty can't open C else he'd reveal the car, so must open B => 100%
})
posterior = prior >> pmfMul >> likelihood >> normalise
posterior >> PP;

PMF(A: 0.333, B: 0.333, C: 0.333)
PMF(A: 0.333, B: 0.000, C: 0.667)


<br>

#### 1.8 Discussion
If the Monty Hall problem is your idea of fun, I have collected a number of similar problems in an article called “All your Bayes are belong to us,” which you can read at http://allendowney.blogspot.com/2011/10/ all-your-bayes-are-belong-to-us.html.

<br>

#### 2.8 Exercises

**Exercise 2.1.** In Section 2.3 I said that the solution to the cookie problem generalizes to the case where we draw multiple cookies with replacement.

But in the more likely scenario where we eat the cookies we draw, the likelihood of each draw depends on the previous draws.

Modify the solution in this chapter to handle selection without replacement. Hint: add instance variables to Cookie to represent the hypothetical state of the bowls, and modify Likelihood accordingly. You might want to define a Bowl object.

<br> 

**Interpretation**

Select a jar, eat some cookies telling me the flavours. This is the one we implement.

Selecting a jar for each cookie would involve keeping tracking of possible combinations of jars and cookies.

In [6]:
@coppertop
def jarLikelihood(jarsStates, flavour) -> L:
    return jarsStates >> collect >> (lambda j: (j.tag, j >> atSlot >> flavour)) >> to >> L

@coppertop
def updateJarModel(jarsStateAndPrior, flavour):
    jarsState, prior = jarsStateAndPrior
    posterior = prior >> pmfMul >> jarLikelihood(jarsState, flavour) >> normalise
    jarsState = jarsState >> collect >> (lambda s: s >> atPut >> flavour >> max(((s >> at >> flavour) - 1, 0)))
    f'Took: {flavour},  posterior: {posterior >> normalise >> formatPmf},  newState: {jarsState}' >> PP
    return (jarsState, posterior)

modelState = [dstruct(V=30, C=10, tag=J1), dstruct(V=20, C=20, tag=J2)]

['C', 'V'] >> inject(_, (modelState, PMF({J1:0.5, J2:0.5})), _) >> updateJarModel;

Took: C,  posterior: PMF(J1: 0.333, J2: 0.667),  newState: [dstruct(V=30, C=9, tag=J1), dstruct(V=20, C=19, tag=J2)]
Took: V,  posterior: PMF(J1: 0.429, J2: 0.571),  newState: [dstruct(V=29, C=9, tag=J1), dstruct(V=19, C=19, tag=J2)]


In [7]:
['V', 'C'] >> inject(_, (modelState, PMF({J1:0.5, J2:0.5})), _) >> updateJarModel;

Took: V,  posterior: PMF(J1: 0.600, J2: 0.400),  newState: [dstruct(V=29, C=10, tag=J1), dstruct(V=19, C=20, tag=J2)]
Took: C,  posterior: PMF(J1: 0.429, J2: 0.571),  newState: [dstruct(V=29, C=9, tag=J1), dstruct(V=19, C=19, tag=J2)]


In [8]:
['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C'] \
  >> inject(_, (modelState, PMF({J1:0.5, J2:0.5})), _) >> updateJarModel;

Took: C,  posterior: PMF(J1: 0.333, J2: 0.667),  newState: [dstruct(V=30, C=9, tag=J1), dstruct(V=20, C=19, tag=J2)]
Took: C,  posterior: PMF(J1: 0.191, J2: 0.809),  newState: [dstruct(V=30, C=8, tag=J1), dstruct(V=20, C=18, tag=J2)]
Took: C,  posterior: PMF(J1: 0.095, J2: 0.905),  newState: [dstruct(V=30, C=7, tag=J1), dstruct(V=20, C=17, tag=J2)]
Took: C,  posterior: PMF(J1: 0.042, J2: 0.958),  newState: [dstruct(V=30, C=6, tag=J1), dstruct(V=20, C=16, tag=J2)]
Took: C,  posterior: PMF(J1: 0.016, J2: 0.984),  newState: [dstruct(V=30, C=5, tag=J1), dstruct(V=20, C=15, tag=J2)]
Took: C,  posterior: PMF(J1: 0.005, J2: 0.995),  newState: [dstruct(V=30, C=4, tag=J1), dstruct(V=20, C=14, tag=J2)]
Took: C,  posterior: PMF(J1: 0.002, J2: 0.998),  newState: [dstruct(V=30, C=3, tag=J1), dstruct(V=20, C=13, tag=J2)]
Took: C,  posterior: PMF(J1: 0.000, J2: 1.000),  newState: [dstruct(V=30, C=2, tag=J1), dstruct(V=20, C=12, tag=J2)]
Took: C,  posterior: PMF(J1: 0.000, J2: 1.000),  newState: [dstr

<br>

#### 3.1 The dice problem

Suppose I have a box of dice that contains a 4-sided die, a 6-sided die, an 8-sided die, a 12-sided die, and a 20-sided die. If you have ever played Dungeons & Dragons, you know what I am talking about.

Suppose I select a die from the box at random, roll it, and get a 6. What is the probability that I rolled each die?

In [9]:
@coppertop
def diceLikelihood(ds, number):
    return ds >> collect >> (lambda d: (d.tag, d >> atOr >> number >> 0.0)) >> to >> L

d4 = (sequence(1, 4) >> uniform)(tag=D4)
d6 = (sequence(1, 6) >> uniform)(tag=D6)
d8 = (sequence(1, 8) >> uniform)(tag=D8)
d12 = (sequence(1, 12) >> uniform)(tag=D12)
d20 = (sequence(1, 20) >> uniform)(tag=D20)

rolled = 6

model = [d4, d6, d8, d12, d20]
PMF({D4:1, D6:1, D8:1, D12:1, D20:1}) >> PP >> pmfMul >> (diceLikelihood(model, rolled) >> PP) >> normalise >> PP;

PMF(D4: 0.200, D6: 0.200, D8: 0.200, D12: 0.200, D20: 0.200)
L(D4: 0.000, D6: 0.167, D8: 0.125, D12: 0.083, D20: 0.050)
PMF(D4: 0.000, D6: 0.392, D8: 0.294, D12: 0.196, D20: 0.118)


In [10]:
@coppertop
def diceUpdate(prior, model, data):
    posterior = prior >> pmfMul >> diceLikelihood(model, data) >> normalise
    return posterior >> PP

[6, 6, 8, 7, 7, 5, 4] >> inject(_, PMF({D4:1, D6:1, D8:1, D12:1, D20:1}), _) >> diceUpdate(_, model, _);

PMF(D4: 0.000, D6: 0.392, D8: 0.294, D12: 0.196, D20: 0.118)
PMF(D4: 0.000, D6: 0.526, D8: 0.296, D12: 0.131, D20: 0.047)
PMF(D4: 0.000, D6: 0.000, D8: 0.735, D12: 0.218, D20: 0.047)
PMF(D4: 0.000, D6: 0.000, D8: 0.818, D12: 0.161, D20: 0.021)
PMF(D4: 0.000, D6: 0.000, D8: 0.876, D12: 0.115, D20: 0.009)
PMF(D4: 0.000, D6: 0.000, D8: 0.916, D12: 0.080, D20: 0.004)
PMF(D4: 0.000, D6: 0.000, D8: 0.943, D12: 0.055, D20: 0.002)


<br>

#### 3.2 The locomotive problem

A railroad numbers its locomotives in order 1..N. One day you see a locomotive with the number 60. Estimate how many locomotives the railroad has.

**Notes**

The railroad has more than 1000 locomotives \
If the railroad has N locomotives then the chance of seeing a particular locomotive is uniformly distributed with P = 1/N \
See http://en.wikipedia.org/wiki/Minimum_mean_square_error

In [11]:
prior = sequence(1, 1000) >> uniform
data = 60

@coppertop
def railroadLikelihood(N, ob):
    return sequence(1, N) \
        >> collect >> (lambda hyp:  (hyp, 0) if hyp < ob else (hyp, 1 / hyp) ) \
        >> to >> L

likelihood = railroadLikelihood(1000, data)
posterior = prior >> pmfMul >> likelihood >> normalise

[posterior \
     >> values \
     >> max, posterior >> at >> 60, posterior >> at >> 59, 
     posterior \
     >> mean
]

ProgrammerError: No constructor defined for type "num"

In [ ]:
### fig = plt.figure(figsize=(5, 5), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(*(posterior >> toSteps));

hmmm... that's quite a heavy tail...

In [ ]:
[250, 500, 1000, 2000, 4000] >> collect >> (lambda N: 
    sequence(1, N) 
        >> uniform
        >> pmfMul 
        >> railroadLikelihood(N, data)
        >> normalise 
        >> mean
)

<br>

#### 3.4 An alternative prior

In fact, the distribution of company sizes tends to follow a power law, as Robert Axtell reports in Science (see http://www.sciencemag.org/content/293/5536/1818.full.pdf)

In [ ]:
@coppertop
def powerLawPrior(n, alpha) -> PMF:
    return sequence(1, 1000) >> collect >> (lambda hyp: (hyp, hyp**(-alpha))) >> to >> PMF

fig = plt.figure(figsize=(5, 5), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(*(1000 >> powerLawPrior(_, 1) >> toSteps));

<br>

#### 3.5 Credible intervals

In [ ]:
observations = [30, 60, 90]
Ns = [250, 500, 1000, 2000, 4000]

Ns >> collect >> (lambda N: 
    powerLawPrior(N, 0.9)
        >> inject(observations, _, _) >> (lambda prior, ob:
            prior
                >> pmfMul 
                >> railroadLikelihood(N, ob)
                >> normalise 
        )
        >> makeFn(lambda pmf: [pmf >> to >> CMF >> quantile(_,0.05), pmf >> mean, pmf >> to >> CMF >> quantile(_,0.95)])
)

<br>

#### 4.1 The Euro problem

A statistical statement appeared in “The Guardian" on Friday January 4, 2002:

When spun on edge 250 times, a Belgian one-euro coin
came up heads 140 times and tails 110. ‘It looks very
suspicious to me,’ said Barry Blight, a statistics lecturer
at the London School of Economics. ‘If the coin were
unbiased, the chance of getting a result as extreme as
that would be less than 7%.’

But do these data give evidence that the coin is biased rather
than fair?


In [ ]:
hypos = sequence(0, 100)

@coppertop
def euroLikelihoodFn(hyp, ob):
    return (hyp, hyp / 100.0) if ob == 'H' else (hyp, 1 - hyp/100)

@coppertop
def euroUpdate(prior, ob):
    like = hypos >> collect >> euroLikelihoodFn(_, ob) >> to >> L
    return prior >> pmfMul >> like

data = ['H'] * 140 + ['T'] * 110

prior = hypos >> uniform     # p that number of heads == hypo

posterior = data >> inject(_, prior, _) >> euroUpdate >> normalise

fig = plt.figure(figsize=(5, 5), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(*(posterior >> toSteps));

In [ ]:
[posterior >> to >> CMF >> quantile(_,0.05), posterior >> mean, posterior >> to >> CMF >> quantile(_,0.95)]

In [ ]:
@coppertop
def triangular(start, nUp, nDown) -> PMF:
    d = {}
    for x in range(0, 51):
        d[x] = x
    for x in range(51, 101):
        d[x] = 100 - x
    return d | PMF
    

In [ ]:
# triangular(10, 10, 10)

IDEAL ERRORS

"hello" >> toCmf

```
Can't find toCmf(str) in:
  toCmf(pmf:(adhoc + (t514 & _numEtAl & _PMF))) in dm.pmf
```

ideally

```
DispatchError: Can't find toCmf(str) in:
  toCmf(pmf:PMF+adhoc) -> any    in dm.pmf[176]
---------------------------------------------------------------------------
Traceback (most recent call last)

<ipython-input-26-8a7f9c734ab9> in <module>
----> 1 "hello" >> toCmf
```